In [ ]:
require 'torch'
require 'xlua'
require 'paths'
require 'image'
py = require 'fb.python'
optnet = require 'optnet'
require('cudnn')
require 'transform'
data = require 'data'

# Retrieve dataset and models parameters

In [ ]:
dataset = 'lsp'

torch.setdefaulttensortype('torch.FloatTensor')
opts = {dataset=dataset, demo=true, res=256}
opts_mem = {inplace=true, reuseBuffers=true, mode='inference'}
activThresh = 0.003

data.checkIntegrity(opts)

# Build model

In [ ]:
model = nil
if opts.dataset == 'mpii' then
  model = torch.load('models/human_pose_mpii.t7')
else 
  model = torch.load('models/human_pose_lsp.t7')
end

model = model:cuda()
cudnn.fastest = true
cudnn.convert(model, cudnn)

optnet.optimizeMemory(model, torch.zeros(1,3,opts.res,opts.res):cuda(), opts_mem)
model:evaluate()

if opts.dataset == 'mpii' then
    dataset = torch.load('dataset/mpii_dataset.t7')
else
    dataset = torch.load('dataset/lsp_dataset.t7')
end

In [ ]:
valDataset = {}
for i=1,#dataset do
  if dataset[i].type == 0 then
    valDataset[#valDataset+1] = dataset[i]
    
    -- If LSP map points to MPII order
    if opts.dataset == 'lsp' then
      local temp_pts = valDataset[#valDataset].points:clone():view(14,2)
      local pts = torch.zeros(16,2)
      pts[{{1,6},{}}] = temp_pts[{{1,6},{}}]
      pts[{{11,16},{}}] = temp_pts[{{7,12},{}}]
      pts[{{9,10},{}}] = temp_pts[{{13,14},{}}]
      valDataset[#valDataset].points = pts
      valDataset[#valDataset]['headSize'] = torch.dist(pts[{{14},{}}],pts[{{3},{}}])
      valDataset[#valDataset]['image'] = string.format("im%04d.jpg",i)
    end
  end
end

ids = torch.range(1,#valDataset)

# Evaluate

In [ ]:
local n = ids:size()[1]
predictions = torch.Tensor(n,16,2)

for i=1,n do
  -- Load and prepare the data
  local img = nil
  if opts.dataset == 'mpii' then
    img = image.load('dataset/mpii_dataset/images/'..valDataset[ids[i]].image)
  else
    img = image.load('dataset/lsp_dataset/images/'..valDataset[ids[i]].image)
  end
  local center = (function() if opts.dataset =='mpii' then return valDataset[ids[i]].center else return torch.Tensor({img:size()[3]/2,img:size()[2]/2}) end end)() 
  local scale = (function() if opts.dataset =='mpii' then return valDataset[ids[i]].scale else return 0.89 end end)() 
  local input = crop(img, center, scale, opts.res)
  input = (function() return input:cuda() end)()
  
  -- Do the forward pass and get the predicitons
  local output = model:forward(input:view(1, 3, opts.res, opts.res))
  
  output = applyFn(function (x) return x:clone() end, output)
  local flippedOut = nil
  flippedOut = model:forward(flip(input:view(1,3,opts.res,opts.res):cuda()))
  flippedOut = applyFn(function (x) return flip(shuffleLR(x)) end, flippedOut)
  output = applyFn(function (x,y) return x:add(y):div(2) end, output, flippedOut):float()

  output[output:lt(0)] = 0
  
  local preds_hm, preds_img = getPreds(output[1], center, scale)
  predictions[{{i},{},{}}] = preds_img
  
  collectgarbage()
end

In [ ]:
local distance = evaluate(predictions,valDataset)
local metrics = calculateMetrics(distance,opts)

print(metrics)

# Preview predictions

In [ ]:
Plot = require('itorch.Plot')

local i = 25

local img = nil
if opts.dataset == 'mpii' then
img = image.load('dataset/mpii_dataset/images/'..valDataset[ids[i]].image)
else
img = image.load('dataset/lsp_dataset/images/'..valDataset[ids[i]].image)
end
local center = (function() if opts.dataset =='mpii' then return valDataset[ids[i]].center else return torch.Tensor({img:size()[3]/2,img:size()[2]/2}) end end)() 
local scale = (function() if opts.dataset =='mpii' then return valDataset[ids[i]].scale else return 0.89 end end)() 
local input = crop(img, center, scale, opts.res)
input = (function() return input:cuda() end)()

-- Do the forward pass and get the predicitons
local output = model:forward(input:view(1, 3, opts.res, opts.res))

output = applyFn(function (x) return x:clone() end, output)
local flippedOut = nil
flippedOut = model:forward(flip(input:view(1,3,opts.res,opts.res):cuda()))
flippedOut = applyFn(function (x) return flip(shuffleLR(x)) end, flippedOut)
output = applyFn(function (x,y) return x:add(y):div(2) end, output, flippedOut):float()

output[output:lt(0)] = 0

local preds_hm, preds_img = getPreds(output[1], center, scale)

for i=1, 16 do
    img[{{},
         {preds_img[{1, i, 2}] - 2, preds_img[{1, i, 2}] + 2},
         {preds_img[{1, i, 1}] - 2, preds_img[{1, i, 1}] + 2}}] = 0
    img[{{2},
         {preds_img[{1, i, 2}] - 2, preds_img[{1, i, 2}] + 2},
         {preds_img[{1, i, 1}] - 2, preds_img[{1, i, 1}] + 2}}] = 1
    for c=1, 3 do
        img[{{2, 3},
             {preds_img[{1, i, 2}] - 1, preds_img[{1, i, 2}] + 1},
             {preds_img[{1, i, 1}] - 1, preds_img[{1, i, 1}] + 1}}] = 0
    end
end
    

itorch.image({img})